In [1]:
!pip install prophet --quiet
!pip install yfinance --quiet

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')



In [3]:
#see the yahoo finance ticker for your stock symbol
stock_symbol = "RELIANCE.NS"

#last 5 years data with interval of 1 day
data = yf.download(tickers=stock_symbol, period='5y', interval='1d')

[*********************100%***********************]  1 of 1 completed


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1235 entries, 2018-05-28 to 2023-05-26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1235 non-null   float64
 1   High       1235 non-null   float64
 2   Low        1235 non-null   float64
 3   Close      1235 non-null   float64
 4   Adj Close  1235 non-null   float64
 5   Volume     1235 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 67.5 KB


In [5]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
data.reset_index(inplace=True)
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
df = data[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
1230,2023-05-22,2435.000000
1231,2023-05-23,2460.649902
1232,2023-05-24,2445.899902
1233,2023-05-25,2433.899902
1234,2023-05-26,2458.000000


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Reliance Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1qymy49u/o_izushu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1qymy49u/gwq3awy1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98118', 'data', 'file=/tmp/tmp1qymy49u/o_izushu.json', 'init=/tmp/tmp1qymy49u/gwq3awy1.json', 'output', 'file=/tmp/tmp1qymy49u/prophet_modelj_r5e3uz/prophet_model-20230528061334.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:13:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:13:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
1595,2024-05-21
1596,2024-05-22
1597,2024-05-23
1598,2024-05-24
1599,2024-05-25


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1595,2024-05-21,2230.153465,1871.507252,2582.035765
1596,2024-05-22,2239.261344,1896.386651,2575.059025
1597,2024-05-23,2239.063765,1876.041538,2585.214417
1598,2024-05-24,2236.238534,1857.887526,2595.982238
1599,2024-05-25,2003.987837,1644.960751,2333.172894


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

2396.8420579069984

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)